# Write selfcontained MODFLOW files
- extract refs and insert their file content to make file selfcontained

In [26]:
import os
import re

In [27]:
model_ws = os.path.join("ReadEnd_WriteCapture","Input","P2R_model")

In [31]:
def replace_refs_and_save_new(file_path, new_suffix="_expanded"):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        # Match lines like: OPEN/CLOSE delr.ref 1 (FREE) -1 delr
        match = re.match(r'^\s*OPEN/CLOSE\s+[\'"]?([^\'"\s]+\.ref)[\'"]?\s+(.*)', line, re.IGNORECASE)
        if match:
            ref_path = match.group(1)
            trailing_header = match.group(2)
            ref_abs_path = os.path.normpath(os.path.join(os.path.dirname(file_path), ref_path))
            if not os.path.isfile(ref_abs_path):
                raise FileNotFoundError(f"Referenced file not found: {ref_abs_path}")

            with open(ref_abs_path, 'r') as ref_file:
                ref_content = ref_file.readlines()

            # Replace with INTERNAL line and keep the rest of the header
            new_lines.append(f"  INTERNAL {trailing_header}\n")
            new_lines.extend(ref_content)
        else:
            new_lines.append(line)

    # Save to new file
    base, ext = os.path.splitext(file_path)
    new_file_path = f"{base}{new_suffix}{ext}"
    with open(new_file_path, 'w') as f:
        f.writelines(new_lines)
    
    print(f"Saved expanded file to: {new_file_path}")

In [32]:
mf_file = os.path.join(model_ws,'P2Rv9.1.dis')
replace_refs_and_save_new(mf_file)

Saved expanded file to: ReadEnd_WriteCapture\Input\P2R_model\P2Rv9.1_expanded.dis
